In [ ]:
import pandapower as pp
import pandapower.networks
import pandas as pd
import plotly.express as px
import numpy as np
np.random.seed(14)
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import os
import time

import matplotlib.pyplot as plt
%matplotlib inline
from pandapower.plotting.plotly import simple_plotly
from pandapower.plotting.plotly import vlevel_plotly
from pandapower.plotting.plotly import pf_res_plotly

from pandapower.timeseries import DFData
from pandapower.timeseries import OutputWriter
from pandapower.timeseries.run_time_series import run_timeseries
from pandapower.control import ConstControl

In [ ]:
net = pp.networks.mv_oberrhein()
net1, net2 = pp.networks.mv_oberrhein(separation_by_sub=True)

In [ ]:
net = net1

In [ ]:
print(f'Number buses: {len(net.bus)}')
print(f'Number loads: {len(net.load)}')
print(f'Number sgens: {len(net.sgen)}')

In [ ]:
# a = [int(i.split(' ')[-1]) for i in net.bus['name']]
bus_indices = net.bus.index

c = {}
l = 0 
s = 0
for i in bus_indices:
	c[i] = [0,0]
	if(i in list(net.load['bus'])):
		c[i][0]+=1
		l+=1
	if(i in list(net.sgen['bus'])):
		c[i][1]+=1
		s+=1
print('{bus index, [load,gen]}')
print(c)

l = [[],[],[],[]]
for k,v in c.items():
	if(v[0]==0 and v[1]==0): #nothing
		l[0].append(k) 
	if(v[0]>0 and v[1]==0): #only load
		l[1].append(k)
	if(v[0]==0 and v[1]>0): #only sgen
		l[2].append(k)
	if(v[0]>0 and v[1]>0): #both
		l[3].append(k)
print('\nNothing: ', l[0])
print('Only load: ', l[1])
print('Only sgen: ', l[2])
print('Both: ', l[3])
print(f'Ratio: Nothing/at least 1: {( len(l[0])/( len(l[0])+len(l[1])+len(l[2])+len(l[3]) ) ):.3f}')

In [ ]:
# a = [int(i.split(' ')[-1]) for i in net.bus['name']]
bus_indices = net.bus.index

c = []
nc = []
l = 0
for i in net.load.to_numpy():
	if(i[1] in bus_indices):
		c.append(i[1])
		l+=1
	else:
		nc.append(i[1])
print(f'Loads connected to something: {c}')
print(f'Loads connected to nothing: {nc}')

print(f'\nLoads-> total: {len(net.load)}, connected to some bus: {l} (ratio: {(l/len(net.load)):.3f})')

In [ ]:
# a = [int(i.split(' ')[-1]) for i in net.bus['name']]
bus_indices = net.bus.index

c = []
nc = []
l = 0
for i in net.sgen.to_numpy():
	if(i[1] in bus_indices):
		c.append(i[1])
		l+=1
	else:
		nc.append(i[1])
print(f'Sgens connected to something: {c}')
print(f'Sgens connected to nothing: {nc}')

print(f'\nSgens-> total: {len(net.sgen)}, connected to some bus: {l} (ratio: {(l/len(net.sgen)):.3f})')

In [ ]:
print(net.load[net.load['scaling'] == 0].count())
print(net.sgen[net.sgen['scaling'] == 0].count()) #All sgen scaling value set to 0!!!! I was wandering why changing sgens time series didn't bring to different results -.-

In [ ]:
#All the scaling values are 0.6
# net.load['scaling'] = 0.6
net.sgen['scaling'] = 0.6
#Can be changed also in the Controllers

net.sgen['sn_mva'] = None

In [ ]:
def num_foreach_element(arr):
	d = {}
	for i in arr.to_numpy():
		i = i[0]
		d[i] = d[i]+1 if i in d.keys() else 1
	return d
	
input_dir = './TimeSeries/1-MVLV-rural-all-0-sw/'
n_timesteps = 4 * 24 * 20

#Loads dataset
profile_load = pd.DataFrame()
n_load = len(net.load)
n_res = len(net.sgen)
# The parameter “sR” generally describes the nominal apparent power of power plants, distributed energy resources and loads
loads = pd.read_csv(f'{input_dir}Load.csv', sep=';')
loads_timeseries = pd.read_csv(f'{input_dir}LoadProfile.csv', sep=';')

RESs = pd.read_csv(f'{input_dir}RES.csv', sep=';')
RESs_timeseries = pd.read_csv(f'{input_dir}RESProfile.csv', sep=';')

#Cases:
# 0: low load, high generation
# 1: normal load and generation
# 2: high load, low generation
case = 0
if(case==-1):
	scale_factor_load = 1.1
	scale_factor_sgen = 2.5
if(case==0):
	scale_factor_load = 0.7
	scale_factor_sgen = 1.4
elif(case==1):
	scale_factor_load = 1
	scale_factor_sgen = 1
elif(case==2):
	scale_factor_load = round(0.8/0.6)
	scale_factor_sgen = 0.1
print(f'Case: {case}, scale load: {scale_factor_load}, scale sgen: {scale_factor_sgen}')
'''
# Papers:
# SimBench—A Benchmark Dataset of Electric Power Systems to Compare Innovative Solutions Based on Power Flow Analysis
# https://www.researchgate.net/profile/Christian-Spalthoff-2/publication/333903713_SimBench_Open_source_time_series_of_power_load_storage_and_generation_for_the_simulation_of_electrical_distribution_grids/links/5d889953458515cbd1b50cef/SimBench-Open-source-time-series-of-power-load-storage-and-generation-for-the-simulation-of-electrical-distribution-grids.pdf?origin=publication_detail
# https://publica.fraunhofer.de/eprints/urn_nbn_de_0011-n-5554297.pdf
Germany standard load profiles(SLPs): Commercial enterprises (G), households (H), agricultural holdings (L) and industrial companies
(BL/BW) were considered as accumulated consumers, while the provided time series for electric
vehicles (EVs) and heat pumps (HPs) were interpreted as individual consumers.
Ending letter: A-C low consumption, M medium, H high
#print(set(loads_timeseries.columns)) 
'''
# net.controller = pd.DataFrame() #Reset controllers
# index_to_select = range(n_load) #np.random.randint(0,len(loads),size=n_load)
index_to_select = np.random.randint(0,len(loads),size=n_load)
loads_type = loads.loc[index_to_select,['profile']]
temp_profile_p = []
temp_profile_q = []
print(f'Chosen load elements by type: {num_foreach_element(loads_type)}')
for l in loads_type.to_numpy():
	val = 0
	noise = np.random.uniform(1-val,1+val,len(loads_timeseries)) #random noise in range [1-val,1+val[ -> change timeseries values by +/-val%
	temp_profile_q.append( loads_timeseries[f'{l[0]}_qload']*noise )
	temp_profile_p.append( loads_timeseries[f'{l[0]}_pload']*noise*0.3 )

#Loads p (in MW) 
profile_load_p = pd.concat(temp_profile_p,axis=1)[:n_timesteps]
profile_load_p.columns = net.load.index
ds_lp = DFData(profile_load_p)
cc_lp = ConstControl(net, 'load', 'p_mw', element_index=net.load.index, profile_name=profile_load_p.columns,
					data_source=ds_lp, scale_factor=scale_factor_load)

#Loads q (in MVar)
profile_load_q = pd.concat(temp_profile_q,axis=1)[:n_timesteps]
profile_load_q.columns = net.load.index
ds_lq = DFData(profile_load_q)
cc_lq = ConstControl(net, 'load', 'q_mvar', element_index=net.load.index, profile_name=profile_load_q.columns,
					data_source=ds_lq, scale_factor=scale_factor_load)

#RES p (in MW)
res_to_add = pd.DataFrame(['WP4','WP7','WP4','WP7','WP4','WP7'], columns=['profile'])
RESs_type = RESs.loc[:n_res-1-len(res_to_add),['profile']]
RESs_type = pd.concat([RESs_type, res_to_add])
temp_profile_p = []
print(f'RES elements by type: {num_foreach_element(RESs_type)}')
for l in RESs_type.to_numpy():
	val = .25
	noise = np.random.uniform(1-val,1+val,len(loads_timeseries)) #random noise in range [1-val,1+val[ -> change timeseries values by +/-val%
	temp_profile_p.append(RESs_timeseries[f'{l[0]}']*noise)
	# temp_profile_p.append(RESs_timeseries[f'{l[0]}_pload']) #Q values are not required

profile_res_p = pd.concat(temp_profile_p,axis=1)[:n_timesteps]
profile_res_p.columns = net.sgen.index
ds_sp = DFData(profile_res_p)
cc_sp = ConstControl(net, 'sgen', 'p_mw', element_index=net.sgen.index, profile_name=profile_res_p.columns,
					data_source=ds_sp, scale_factor=scale_factor_sgen)

In [ ]:
figsize=(20, 3)
fig = plt.figure(figsize=figsize)
(cc_lp.data_source.df*cc_lp.scale_factor).sum(axis=1).plot(label='load p',ylabel='mw')
(cc_lq.data_source.df*cc_lq.scale_factor).sum(axis=1).plot(label='load q',ylabel='mw')
(cc_sp.data_source.df*cc_sp.scale_factor).sum(axis=1).plot(label='sgen')
# profile_res_p.loc[:,['PV5', 'PV8', 'PV6']].sum(axis=1).plot(label='sgen PV')
# profile_res_p.loc[:,['WP4','WP7']].sum(axis=1).plot(label='sgen WP')
plt.legend()
plt.show()

In [ ]:
df = profile_load_p
# df =  df.loc[:,~df.columns.duplicated()]
df.boxplot(rot=45,figsize=figsize)
plt.ylabel('mw (?)')

In [ ]:
time_steps = range(0,n_timesteps)

output_dir = os.path.join(input_dir,"Results", str(case))
ow = OutputWriter(net, time_steps, output_path=output_dir, output_file_type=".csv", log_variables=list())

#Save time series (output)
# these variables are saved to the harddisk after / during the time series loop
ow.log_variable('res_load', 'p_mw')
ow.log_variable('res_load', 'q_mvar')
ow.log_variable('res_bus', 'vm_pu')
# ow.log_variable('res_bus', 'va_degree')
ow.log_variable('res_line', 'loading_percent')
# ow.log_variable('res_line', 'i_ka')

print('Time steps: ',len(time_steps), '. Num Loads: ', net.load.index.shape, '. Load p and q: ', profile_load_p.shape, profile_load_q.shape, '. Num RESs: ', net.sgen.index.shape, '. RESs p: ',profile_res_p.shape)
t1 = time.time()
run_timeseries(net,time_steps)
t2 = time.time()
print(f'Simulation time : {(t2-t1):.2f} s')

#Save time series (input)
path = os.path.join(output_dir, "loads_p.csv")
(cc_lp.data_source.df*cc_lp.scale_factor).to_csv(path)
path = os.path.join(output_dir, "loads_q.csv")
(cc_lq.data_source.df*cc_lq.scale_factor).to_csv(path)
path = os.path.join(output_dir, "RESs_p.csv")
(cc_sp.data_source.df*cc_sp.scale_factor).to_csv(path)
t3 = time.time()
print(f'Saving files time: {(t3-t2):.2f} s')

In [ ]:
width = 1000
fig = px.line(cc_lp.data_source.df*cc_lp.scale_factor,x=time_steps, y=profile_load_p.columns, width=width, height=400)
fig.show()
fig = px.line(cc_lq.data_source.df*cc_lq.scale_factor,x=time_steps, y=profile_load_q.columns, width=width, height=400)
fig.show()
fig = px.line(cc_sp.data_source.df*cc_sp.scale_factor,x=time_steps, y=profile_res_p.columns, width=width, height=400)
fig.show()

In [ ]:
def plot_df(df,title='',y_axis='',file_name=''):
	fig = px.line(df,x=time_steps, y=df.columns, width=1200, height=800)
	fig.update_layout(title=title,
					xaxis_title='time steps',
					yaxis_title=y_axis)
	if(file_name):
		fig.write_html(file_name)
		print(f'Saved {title} in {file_name}')

output_dir = os.path.join(input_dir,"Results", str(case))

# voltage results
vm_pu_file = os.path.join(output_dir, "res_bus", "vm_pu.csv")
vm_pu = pd.read_csv(vm_pu_file, index_col=0, sep=';')
vm_pu = vm_pu.drop(columns='58' ) #External grid
# plot_df(vm_pu,'buses voltage magnitude', 'bus vm [pu]', os.path.join(output_dir, "Plots", "bus vm.html"))

# # line loading resulcsvts
ll_file = os.path.join(output_dir, "res_line", "loading_percent.csv")
line_loading = pd.read_csv(ll_file, index_col=0, sep=';')
# plot_df(line_loading,'line_loading', 'line_loading [%]', os.path.join(output_dir, "Plots", "line load.html"))

# # load results
load_file = os.path.join(output_dir, "res_load", "p_mw.csv")
load = pd.read_csv(load_file, index_col=0, sep=';')
# plot_df(load,'load active power', 'p [MW]', os.path.join(output_dir, "Plots", "load.html"))

In [ ]:
df = pd.DataFrame()
df['max'] = vm_pu.max(axis=1)
df['min'] = vm_pu.min(axis=1)
ch = []
cl = []

for index, row in df.iterrows():
	if(row['max']>1.05):
		ch.append('r')
	else:
		ch.append('b')
	if(row['min']<0.95):
		cl.append('r')
	else:
		cl.append('b')
		
df['over'] = ch
df['under'] = cl
# # d = {True: 'Voltage problem', False: 'Normal condition'}
# # df = df.replace(d)
# fig = px.scatter(df,x=df.index,y='max',color='over', width=1000, height=400)
# # fig.update_yaxes(type='category')
# fig.update_traces(marker_size=5)
# fig.update_layout(
# 	title="",
# 	xaxis_title="Time steps",
# 	yaxis_title="max bus vm [pu]",
# 	legend_title="")

In [ ]:
fig = plt.figure(figsize=(16,5))
s = 27
ma = plt.scatter(x=df.index,y=df['max'],c=df['over'],s=s, marker='^')
mi = plt.scatter(x=df.index,y=df['min'],c=df['under'],s=s, marker='v')

plt.scatter(x=[],y=[],c=['r'], marker='^', s=s, label='Maxs overvoltages')
plt.scatter(x=[],y=[],c=['b'], marker='^', s=s, label='Maxs normal')
plt.scatter(x=[],y=[],c=['r'], marker='v', s=s, label='Mins undervoltages')
plt.scatter(x=[],y=[],c=['b'], marker='v', s=s, label='Mins normal')
plt.legend()

plt.show()

In [ ]:
#Plot network and loadings

bus = 58
fig = simple_plotly(net, bus_size=5, ext_grid_size=10)
fig.add_trace(px.scatter(x=[net.bus_geodata.loc[bus, 'x']], y=[net.bus_geodata.loc[bus, 'y']],color=['r'],size=[10]).data[0])
# _ = vlevel_plotly(net, bus_size=5, ext_grid_size=10)
# fig = pf_res_plotly(net, bus_size=8)
# fig.write_html(f"images/MVOberrhein/Half2.html")
# fig.write_image(f"images/MVOberrhein/Half2.png")

In [ ]:
print(vm_pu[58]) #constant voltage?
print(net.bus.loc[95])
print(net.load[net.load['bus']==58]) #no load in bus 58

In [ ]:
# vm_pu.plot(label="vm_pu", figsize=figsize)

In [ ]:
#Naive version
def classification_dataset(df):
	labels = []
	a = 0
	acceptance_range = 0.1
	for i in df.to_numpy():
		if(np.any(i>1+acceptance_range/2) or np.any(i<1-acceptance_range/2) ):#or np.any(i<1-acceptance_range/2)
			labels.append(1)
			a+=1
		else:
			labels.append(0)
	print(f'Number of problems: {a}, total number: {len(df)}, ratio: {(a/len(df)*100):.1f}%')
	return pd.DataFrame(labels,columns=['Label'])

vm_pu_classification = classification_dataset(vm_pu)

In [ ]:
x = load
y = vm_pu_classification

In [ ]:
# x.columns = [f'l_{name}' for name in x.columns]
# y.columns = [f'bvm_{name}' for name in y.columns]
df = pd.concat([x], axis=1)

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

train_mean = train_df.mean()
train_std = train_df.std()

if(True):
	train_df = pd.concat( [(train_df - train_mean) / train_std,y[0:int(n*0.7)]], axis=1)
	val_df = pd.concat( [(val_df - train_mean) / train_std,y[int(n*0.7):int(n*0.9)]], axis=1)
	test_df = pd.concat( [(test_df - train_mean) / train_std,y[int(n*0.9):]], axis=1)
else:
	train_df = pd.concat( [(train_df), y[0:int(n*0.7)]], axis=1)
	val_df = pd.concat( [(val_df), y[int(n*0.7):int(n*0.9)]], axis=1)
	test_df = pd.concat( [(test_df), y[int(n*0.9):]], axis=1)

print(train_df.shape,val_df.shape,test_df.shape)

In [ ]:
a = y[int(n*0.9):]
print(a.sum(),a.sum()/a.count())

In [ ]:
class WindowGenerator():
	def __init__(self, input_width, label_width, shift,
				train_df=train_df, val_df=val_df, test_df=test_df,
				all_columns =None, training_columns=None, label_columns=None):
		# Store the raw data.
		self.train_df = train_df
		self.val_df = val_df
		self.test_df = test_df

		self.all_columns = all_columns
		self.column_indices = {name: i for i, name in enumerate(self.all_columns)}
		print('All columns: ', self.column_indices)

		self.training_columns = training_columns
		# print('Train columns: ', self.column_indices_t)

		self.label_columns = label_columns
		# print('Label columns: ', self.column_indices_l)

		# Work out the window parameters.
		self.input_width = input_width
		self.label_width = label_width
		self.shift = shift

		self.total_window_size = input_width + shift

		self.input_slice = slice(0, input_width)
		self.input_indices = np.arange(self.total_window_size)[self.input_slice]

		self.label_start = self.total_window_size - self.label_width
		self.labels_slice = slice(self.label_start, None)
		self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

	def __repr__(self):
		return '\n'.join([
			f'Total window size: {self.total_window_size}',
			f'Input indices: {self.input_indices}',
			f'Label indices: {self.label_indices}',
			f'Label column name(s): {self.label_columns}'])

	def split_window(self, features):
		inputs = features[:, self.input_slice, :]
		indices = [self.column_indices[c] for c in self.training_columns]
		# inputs = tf.stack( [inputs[:, :, self.all_columns.index(name)] for name in self.training_columns], axis=-1)
		inputs = tf.stack( [inputs[:, :, i] for i in indices], axis=-1)
		indices = [self.column_indices[c] for c in self.training_columns]

		# labels = features[:, self.labels_slice, :]
		labels = features[:, self.labels_slice, :]
		indices = [self.column_indices[c] for c in self.label_columns]

		labels = tf.stack( [labels[:, :, i] for i in indices], axis=-1)
		# labels = tf.stack( l, axis=-1)


		# Slicing doesn't preserve static shape information, so set the shapes
		# manually. This way the `tf.data.Datasets` are easier to inspect.
		inputs.set_shape([None, self.input_width, None])
		# labels.set_shape([None, self.label_width, None])
		labels = tf.reshape(labels, [-1,self.label_width])

		return inputs, labels

	def plot(self, model=None):
		predictions = []
		labels = []
		a = 0
		for i,l in self.test:
			if model is not None:
				predictions.append(list(tf.reshape(model(i),[-1]).numpy()))
			a+=1
			labels.append(list(tf.reshape(l,[-1]).numpy()))
		predictions = [item for sublist in predictions for item in sublist]
		predictions = np.round(predictions)*0.9 # for visualization purposes
		labels = [item for sublist in labels for item in sublist]


		x = range(len(predictions))
		plt.figure(figsize=(16,6))
		s = 1
		plt.scatter(x, predictions,s=s)
		plt.scatter(x, labels,s=s)
		plt.show()
		return predictions,labels

	def make_dataset(self, data):
		data = np.array(data, dtype=np.float32)
		ds = tf.keras.utils.timeseries_dataset_from_array(
			data=data,
			targets=None,
			sequence_length=self.total_window_size,
			sequence_stride=1,
			shuffle=True,
			batch_size=32,)
			
		ds = ds.map(self.split_window)

		return ds

	@property
	def train(self):
		return self.make_dataset(self.train_df)

	@property
	def val(self):
		return self.make_dataset(self.val_df)

	@property
	def test(self):
		return self.make_dataset(self.test_df)

	@property
	def example(self):
		"""Get and cache an example batch of `inputs, labels` for plotting."""
		result = getattr(self, '_example', None)
		if result is None:
			# No example batch was found, so get one from the `.train` dataset
			result = next(iter(self.test))
		# And cache it for next time
		self._example = result
		return result

In [ ]:
all_columns = train_df.columns
training_columns = x.columns
label_columns = y.columns
input_window = 2
output_window = 1
shift = output_window
dwg = WindowGenerator(input_window,output_window,shift,all_columns=all_columns,training_columns=training_columns,label_columns=label_columns)

In [ ]:
for example_inputs, example_labels in dwg.train.take(1):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  print(f'Labels shape (batch, time, features): {example_labels.shape}')


In [ ]:
dense = tf.keras.Sequential([
	tf.keras.layers.Flatten(),
	tf.keras.layers.Dense(units=64*3, activation='relu'),
	tf.keras.layers.Dense(units=64*2, activation='relu'),
	tf.keras.layers.Dense(units=output_window*len(label_columns), activation='sigmoid'),
	# tf.keras.layers.Reshape([output_window, len(label_columns)])
])

In [ ]:
MAX_EPOCHS = 50

monitor = 'val_recall'
def compile_and_fit(model, window, patience=6):
	early_stopping = tf.keras.callbacks.EarlyStopping(monitor=monitor,
													patience=patience,
													mode='max',
													verbose=1)

	model.compile(loss=tf.losses.BinaryCrossentropy(),
				optimizer=tf.optimizers.Adam(),
				metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
							tf.keras.metrics.Precision(name='precision'),
							tf.keras.metrics.Recall(name='recall')]
				)

	history = model.fit(window.train, epochs=MAX_EPOCHS,
						validation_data=window.val,
						callbacks=[])#early_stopping
	return history

In [ ]:
history = compile_and_fit(dense, dwg)

In [ ]:
x_a = np.arange(0,MAX_EPOCHS)
# x = np.arange(0,42)
plt.plot(
    x_a,
    history.history['loss'], label='Loss'
)
plt.plot(
    x_a,
    history.history['accuracy'], label='Accuracy'
)
plt.plot(
    x_a,
    history.history['precision'], label='Precision'
)
plt.plot(
    x_a,
    history.history['recall'], label='Recall'
)
plt.title('Evaluation metrics', size=20)
plt.xlabel('Epoch', size=14)
plt.legend();

In [ ]:
x_a = np.arange(0,MAX_EPOCHS)
# x = np.arange(0,42)
# plt.plot(
#     x_a,
#     history.history['val_loss'], label='Val Loss'
# )
plt.plot(
    x_a,
    history.history['val_accuracy'], label='Val Accuracy'
)
plt.plot(
    x_a,
    history.history['val_precision'], label='Val Precision'
)
plt.plot(
    x_a,
    history.history['val_recall'], label='Val Recall'
)
plt.title('Evaluation metrics', size=20)
plt.xlabel('Epoch', size=14)
plt.legend();

In [ ]:
a,b = dwg.plot(dense)

In [ ]:
dense.predict(example_inputs)

In [ ]:
results = dense.evaluate(dwg.test)
print("test loss, test acc:", results)
